In [1]:
import json
import os
import glob

import psycopg2

In [2]:
# Define the competitions and seasons you're interested in
competitions_and_seasons = {
    'La Liga': ['2020/2021', '2019/2020', '2018/2019'],
    'Premier League': ['2003/2004']
}

# Initialize an empty list to store the filtered data
matches_data = []

# Define the base directory of the local repository for matches
matches_base_dir = 'E:\\database\\2024\\Laliga EPL Queries - Javierkoren\\open-data\\data\\matches\\'

# Define the base directory of the local repository for lines and events
base_dir = 'E:\\database\\2024\\Laliga EPL Queries - Javierkoren\\open-data\\data\\'

In [3]:
# Get the list of subfolders in matches folder
subfolders = [f.path for f in os.scandir(matches_base_dir) if f.is_dir()]

# Loop through each subfolder in matches folder
for subfolder in subfolders:
    # Get the list of JSON files in the subfolder
    json_files = glob.glob(os.path.join(subfolder, '*.json'))
    
    # Loop through each JSON file
    for json_file in json_files:
        # Open and read the JSON file with 'utf-8' encoding
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Filter out the competitions and seasons
        matches_data += [item for item in data if item['competition']['competition_name'] in competitions_and_seasons and item['season']['season_name'] in competitions_and_seasons[item['competition']['competition_name']]]


In [4]:
# Initialize an empty list to store the combined data
lineups_data = []
events_data = []

# Loop through each item in the filtered data
for item in matches_data:
    # Get the match_id
    match_id = item['match_id']
    
    # Define the paths to the lineup and events JSON files for this match_id
    lineup_file = os.path.join(base_dir, 'lineups', f'{match_id}.json')
    events_file = os.path.join(base_dir, 'events', f'{match_id}.json')
    
    # Open and read the lineup JSON file with 'utf-8' encoding
    with open(lineup_file, 'r', encoding='utf-8') as f:
        lineup_data = json.load(f)
    
    # Open and read the events JSON file with 'utf-8' encoding
    with open(events_file, 'r', encoding='utf-8') as f:
        events_data = json.load(f)
    
    # Combine the item, lineup_data, and events_data into a new dictionary and append it to the combined_data list
    lineups_data.append({**item, 'lineup': lineup_data})
    events_data.append({**item, 'events': events_data})

In [5]:
# Loop through each match in matches_data

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname='statsbomb_football',
    user='postgres',
    password='Password',
    host='localhost'
)

cur = conn.cursor()
for match in matches_data:
    # Check if 'competition' and 'home_team' and 'away_team' are in match
    if 'competition' in match and 'home_team' in match and 'away_team' in match:
        competition = match['competition']
        home_team = match['home_team']
        away_team = match['away_team']
                
        # Insert competition data
        competition_id = competition['competition_id']
        competition_name = competition['competition_name']
        country_name = competition['country_name']        
        cur.execute("INSERT INTO competition (competition_id, competition_name, country_name) VALUES (%s, %s, %s)  ON CONFLICT (competition_id) DO NOTHING", (competition_id, competition_name, country_name))
        
        # Insert season data
        if 'season' in match:
            season = match['season']
            season_id = season['season_id']
            season_name = season['season_name']
            cur.execute("INSERT INTO season (season_id, season_name) VALUES (%s, %s)  ON CONFLICT (season_id) DO NOTHING", (season_id, season_name))

        # Insert home team data
        home_team_id = home_team['home_team_id']
        home_team_name = home_team['home_team_name']
        home_team_gender = home_team['home_team_gender']
        country_id = home_team['country']['id']
        country_name = home_team['country']['name']        
        cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name))        
        cur.execute("INSERT INTO team (team_id, team_name, team_gender, country_id) VALUES (%s, %s, %s, %s) ON CONFLICT (team_id) DO NOTHING", (home_team_id, home_team_name, home_team_gender, country_id))
        
        # Insert away team data
        away_team_id = away_team['away_team_id']
        away_team_name = away_team['away_team_name']
        away_team_gender = away_team['away_team_gender']
        country_id = away_team['country']['id']
        country_name = away_team['country']['name']   
        cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name)) 
        cur.execute("INSERT INTO team (team_id, team_name, team_gender, country_id) VALUES (%s, %s, %s, %s) ON CONFLICT (team_id) DO NOTHING", (away_team_id, away_team_name, away_team_gender, country_id))
        
        # Insert manager data for home team
        for manager in home_team.get('managers', []):
            manager_id = manager['id']
            manager_name = manager['name']
            manager_dob = manager['dob']
            manager_nickname = manager['nickname']
            country_id = manager['country']['id']
            country_name = manager['country']['name']   
        cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name)) 
        cur.execute("INSERT INTO manager (manager_id, manager_name, manager_nickname, manager_dob, country_id) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (manager_id) DO NOTHING", (manager_id, manager_name, manager_nickname, manager_dob, country_id))
            
        # Insert manager data for away team
        for manager in away_team.get('managers', []):
            manager_id = manager['id']
            manager_name = manager['name']
            manager_dob = manager['dob']
            manager_nickname = manager['nickname']
            country_id = manager['country']['id']
            country_name = manager['country']['name']   
        
        cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name)) 
        cur.execute("INSERT INTO manager (manager_id, manager_name, manager_nickname, manager_dob, country_id) VALUES (%s, %s, %s, %s, %s) ON CONFLICT (manager_id) DO NOTHING", (manager_id, manager_name, manager_nickname, manager_dob, country_id))

        # Insert referee data
        if 'referee' in match:
            referee = match['referee']
            referee_id = referee['id']
            referee_name = referee['name']
            country_id = referee['country']['id']
            country_name = referee['country']['name'] 
            cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name)) 
            cur.execute("INSERT INTO referee (referee_id, referee_name, country_id) VALUES (%s, %s, %s)  ON CONFLICT (referee_id) DO NOTHING", (referee_id, referee_name, country_id))
        
        # Insert stadium data
        if 'stadium' in match:
            stadium = match['stadium']
            stadium_id = stadium['id']
            stadium_name = stadium['name']
            country_id = stadium['country']['id']
            country_name = stadium['country']['name'] 
            cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name)) 
            cur.execute("INSERT INTO stadium (stadium_id, stadium_name, country_id) VALUES (%s, %s, %s)  ON CONFLICT (stadium_id) DO NOTHING", (stadium_id, stadium_name, country_id))

        # Insert stadium data
        if 'competition_stage' in match:
            competition_stage = match['competition_stage']
            competition_stage_id = competition_stage['id']
            competition_stage_name = competition_stage['name']
            cur.execute("INSERT INTO competition_stage (competition_stage_id, competition_stage_name) VALUES (%s, %s)  ON CONFLICT (competition_stage_id) DO NOTHING", (competition_stage_id, competition_stage_name))


In [6]:
# Loop through each match in matches_data
for match in matches_data:
    if 'competition' in match and 'home_team' in match and 'away_team' in match and 'season' in match and 'stadium' in match and 'competition_stage' in match:
        match_id = match['match_id']
        match_date = match['match_date']
        kick_off = match['kick_off']
        competition_id = match['competition']['competition_id']
        season_id = match['season']['season_id']
        stadium_id = match['stadium']['id']
        home_team_id = match['home_team']['home_team_id']
        home_team_group = match['home_team']['home_team_group']
        home_team_score = match['home_score']
        away_team_id = match['away_team']['away_team_id']
        away_team_group = match['away_team']['away_team_group']
        away_team_score = match['away_score']
        match_status = match['match_status']
        match_week = match['match_week']
        competition_stage_id = match['competition_stage']['id']

        home_team = match['home_team']
        away_team = match['away_team']
        
        # Insert match data
        cur.execute("INSERT INTO match (match_id, match_date, kick_off, competition_id, season_id, stadium_id, referee_id, home_team_id, home_team_group, home_team_score, away_team_id, away_team_group, away_team_score, match_status, match_week, competition_stage_id) "
                    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (match_id) DO NOTHING", 
                    (match_id, match_date, kick_off, competition_id, season_id, stadium_id, referee_id, home_team_id, home_team_group, home_team_score, away_team_id, away_team_group, away_team_score, match_status, match_week, competition_stage_id))

        # Insert home team manager data
        for manager in home_team.get('managers', []):
            home_team_manager_id = manager['id']
            cur.execute("SELECT COUNT(*) FROM manager WHERE manager_id = %s", (home_team_manager_id,))
            manager_count = cur.fetchone()[0]
            if manager_count > 0:
                cur.execute("UPDATE match SET home_team_manager_id = %s WHERE match_id = %s", (home_team_manager_id,match_id,))
        
        # Insert away team manager data
        for manager in away_team.get('managers', []):
            away_team_manager_id = manager['id']
            cur.execute("SELECT COUNT(*) FROM manager WHERE manager_id = %s", (away_team_manager_id,))
            manager_count = cur.fetchone()[0]
            if manager_count > 0:
                cur.execute("UPDATE match SET away_team_manager_id = %s WHERE match_id = %s", (away_team_manager_id, match_id))

        for lineups in lineup_data:
            # check if team exists
            if 'team' in lineups:
                    team = lineups['team']
                    team_id = team['team_id']
                    team_name = team['team_name']
                    cur.execute("INSERT INTO team (team_id, team_name) VALUES (%s, %s) ON CONFLICT (team_id) DO NOTHING", (team_id, team_name))
                
            for player in lineups['lineup']:
                player_id = player['player_id']
                player_name = player['player_name']
                player_nickname = player['player_nickname'] 
                jersey_number = player['jersey_number'] 
                country_id = player['country']['id']
                country_name = player['country']['name']   
               
                cur.execute("INSERT INTO country (country_id, country_name) VALUES (%s, %s) ON CONFLICT (country_id) DO NOTHING", (country_id, country_name))
                cur.execute("INSERT INTO player (player_id, player_name, player_nickname, jersey_number, country_id) VALUES (%s, %s, %s, %s, %s)  ON CONFLICT (player_id) DO NOTHING", (player_id, player_name, player_nickname, jersey_number, country_id))

            # Insert home team data
                player_team_id = lineups['team_id']
                cur.execute("INSERT INTO lineup (team_id, match_id, player_id) VALUES (%s, %s, %s) ON CONFLICT (team_id, match_id, player_id) DO NOTHING", (player_team_id, match_id, player_id))        

        # Loop through each match in events_data for passes
        for event in events_data:    
            if 'pass' in event:
                
                pass_id = event['id']
                player = event['player']
        
                player_id = player['id']
                player_name = player['name']
                pass_e = event['pass']
                
                if 'recipient' in pass_e:
                    recipient = pass_e['recipient']
                
                length = pass_e['length']
                angle = pass_e['angle']
                pass_height = pass_e['height']
                end_location = pass_e['end_location']
                assisted_shot_id = None
                backheel = None
                deflected = None
                miscommunication = None
                pass_cross = None
                cut_back = None
                switch = None
                shot_assist = None
                goal_assist = None
                body_part = None
                pass_type = None
                pass_outcome = None
                pass_technique = None
                
                if 'assisted_shot_id' in pass_e:
                    assisted_shot_id  = pass_e['assisted_shot_id']
                if 'backheel' in pass_e:
                    backheel  = pass_e['backheel']
                if 'deflected' in pass_e:
                    deflected  = pass_e['deflected']
                if 'miscommunication' in pass_e:
                    miscommunication  = pass_e['miscommunication']
                if 'cross' in pass_e:
                    pass_cross  = pass_e['cross']
                if 'cut_back' in pass_e:
                    cut_back  = pass_e['cut_back']
                if 'switch' in pass_e:
                    switch  = pass_e['switch']
                if 'shot_assist' in pass_e:
                    shot_assist  = pass_e['shot_assist']
                if 'goal_assist' in pass_e:
                    goal_assist  = pass_e['goal_assist']
                if 'body_part' in pass_e:
                    body_part  = pass_e['body_part']
                if 'type' in pass_e:
                    pass_type  = pass_e['type']
                if 'technique' in pass_e:
                    pass_technique  = pass_e['technique']
                if 'outcome' in pass_e:
                    pass_outcome  = pass_e['outcome']
                pass_outcome_id = pass_outcome['id'] if pass_outcome is not None else 1000
                pass_outcome_name = pass_outcome['name'] if pass_outcome is not None else 'Completed'
                
                recipient_id = recipient['id']

                if pass_technique is not None:
                    pass_technique_id = pass_technique['id']
                    pass_technique_name = pass_technique['name']
                else:
                    pass_technique_id = 1000
                    pass_technique_name = "Unknown"
                
                pass_height_id = pass_height['id']
                pass_height_name = pass_height['name'] 

                if pass_type is not None:
                    pass_type_id = pass_type['id']
                    pass_type_name = pass_type['name']
                else:
                    pass_type_id = 1000
                    pass_type_name = "standard pass in the run of play" 

                cur.execute("INSERT INTO pass_outcome (pass_outcome_id, pass_outcome_name) "
                            "VALUES (%s, %s) ON CONFLICT (pass_outcome_id) DO NOTHING",(pass_outcome_id, pass_outcome_name))

                cur.execute("INSERT INTO pass_technique (pass_technique_id, pass_technique_name) "
                            "VALUES (%s, %s) ON CONFLICT (pass_technique_id) DO NOTHING",(pass_technique_id, pass_technique_name))
                
                cur.execute("INSERT INTO pass_height (height_id, height_name) "
                            "VALUES (%s, %s) ON CONFLICT (height_id) DO NOTHING",(pass_height_id, pass_height_name))

                cur.execute("INSERT INTO pass_type (pass_type_id, pass_type_name) "
                            "VALUES (%s, %s) ON CONFLICT (pass_type_id) DO NOTHING",(pass_type_id, pass_type_name))

                                
                cur.execute("INSERT INTO pass (pass_id, match_id, player_id, recipient_id, pass_technique_id, length, angle, height_id, end_location, assisted_shot_id, backheel, deflected, miscommunication, pass_cross, cut_back, switch, shot_assist, goal_assist, pass_type_id, pass_outcome_id) "
                            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (pass_id) DO NOTHING",
                            (pass_id, match_id, player_id, recipient_id, pass_technique_id, length, angle, pass_height_id, end_location, assisted_shot_id, backheel, deflected, miscommunication, pass_cross, cut_back, switch, shot_assist, goal_assist, pass_type_id, pass_outcome_id))
        # Loop through each match in matches_data
        for event in events_data:    
            if 'dribble' in event:
                
                dribble_id = event['id']
                player = event['player']
        
                player_id = player['id']
                player_name = player['name']
                dribble = event['dribble']
                    
                nutmeg = None
                overrun = None
                no_touch = None
                
                if 'nutmeg' in dribble:
                    nutmeg  = dribble['nutmeg']
                if 'overrun' in dribble:
                    overrun  = dribble['overrun']
                if 'notouch' in dribble:
                    no_touch  = dribble['notouch']
                    
                dribble_outcome = dribble['outcome']
                dribble_outcome_id = dribble_outcome['id']
                dribble_outcome_name = dribble_outcome['name']
                
                # Insert outcome data
                cur.execute("INSERT INTO dribble_outcome (dribble_outcome_id, dribble_outcome_name) "
                            "VALUES (%s, %s) ON CONFLICT (dribble_outcome_id) DO NOTHING",(dribble_outcome_id, dribble_outcome_name))
                
                # Insert dribbles data
                cur.execute("INSERT INTO dribble (dribble_id, match_id, player_id, nutmeg, overrun, No_Touch, dribble_outcome_id) "
                            "VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (dribble_id) DO NOTHING",(dribble_id, match_id, player_id, nutmeg, overrun, no_touch, dribble_outcome_id))
                
        for event in events_data:    
            if 'type' in event:
                event_type = event['type']
                        
                if 'name' in event_type:
                    eventtype_name = event_type['name']
                if 'id' in event_type:
                    eventtype_id = event_type['id']
                
                if eventtype_name == "Dribbled Past":
                    dribbled_past_id = event['id']
                    player = event['player']
                    player_id = player['id']
                    
                    # Insert dribbles data
                    cur.execute("INSERT INTO dribbled_past (dribbled_past_id, match_id, player_id) "
                                "VALUES (%s, %s, %s) ON CONFLICT (dribbled_past_id) DO NOTHING",
                                (dribbled_past_id, match_id, player_id))

                
        # Loop through each match in matches_data
        for event in events_data:    
            if 'shot' in event:
                
                shot_id = event['id']
                player = event['player']
        
                player_id = player['id']
                player_name = player['name']
                shot = event['shot']
                    
                aerial_won = None
                follows_dribble = None
                first_time = None
                open_goal = None
                deflected = None
                key_pass_id = None
                
                if 'aerial_won' in shot:
                    aerial_won  = shot['aerial_won']
                if 'follows_dribble' in shot:
                    follows_dribble  = shot['follows_dribble']
                if 'first_time' in shot:
                    first_time  = shot['first_time']
                if 'open_goal' in shot:
                    open_goal  = shot['open_goal']
                if 'deflected' in shot:
                    deflected = shot['deflected']
                if 'key_pass_id' in shot:
                     key_pass_id = shot['key_pass_id']
                    
                end_location = shot['end_location']
                statsbomb_xg  = shot['statsbomb_xg']
                
                body_part = shot['body_part']
                technique = shot['technique']
                outcome = shot['outcome']
                shot_type = shot['type']
                
                if first_time in shot:
                    first_time = shot['first_time']
        
                outcome_id = outcome['id']
                outcome_name = outcome['name']
        
                technique_id = technique['id']
                technique_name = technique['name']
        
                body_part_id = body_part['id']
                body_part_name = body_part['name']
                
                shot_type_id = shot_type['id']
                shot_type_name = shot_type['name']
                               
                # Insert outcome data
                cur.execute("INSERT INTO shot_outcome (outcome_id, outcome_name) "
                            "VALUES (%s, %s) ON CONFLICT (outcome_id) DO NOTHING",(outcome_id, outcome_name))
                # Insert technique data
                cur.execute("INSERT INTO shot_technique (technique_id, technique_name) "
                            "VALUES (%s, %s) ON CONFLICT (technique_id) DO NOTHING",(technique_id, technique_name))
        
                 # Insert body_part data
                cur.execute("INSERT INTO body_part (body_part_id, body_part_name) "
                            "VALUES (%s, %s) ON CONFLICT (body_part_id) DO NOTHING",(body_part_id, body_part_name))
        
                # Insert type data
                cur.execute("INSERT INTO shot_type (shot_type_id, shot_type_name) "
                            "VALUES (%s, %s) ON CONFLICT (shot_type_id) DO NOTHING",(shot_type_id, shot_type_name))
        
                cur.execute("INSERT INTO shot (shot_id, match_id, player_id, key_pass_id, end_location, aerial_won, follows_dribble, first_time, open_goal, statsbomb_xg, deflected, technique_id, body_part_id, shot_type_id, outcome_id) "
                            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (shot_id) DO NOTHING",
                            (shot_id, match_id, player_id, key_pass_id, end_location, aerial_won, follows_dribble, first_time, open_goal, statsbomb_xg, deflected, technique_id, body_part_id, shot_type_id, outcome_id))
               

# Commit the changes
conn.commit()


In [7]:
# Close the cursor and connection
cur.close()
conn.close()